In [1]:

import numpy as np
import pandas as pd
from typing import List
# import matplotlib.pyplot as plt

from lmfit import Minimizer, minimize, Parameters, fit_report

from lib.transformations import (euler_matrix, 
                                 euler_from_matrix, 
                                 scale_from_matrix,
)
from lib.utils import convert_to_homogeneous, print_results
from lib.io import read_data_to_df


def compute_residuals(
    params: Parameters,
    x0: np.ndarray, 
    x1: np.ndarray, 
    weights: np.ndarray = None,
    prior_covariance_scale: float = None, 
) -> np.ndarray: 
    ''' 3D rototranslation with scale factor

    X1_ = T_ + m * R * X0_

    Inputs: 
    - x0 (np.ndarray): Points in the starting reference system
    - x1 (np.ndarray): Points in final reference system 
    - weights (np.ndarray, defult = None): weights (e.g., inverse of a-priori observation uncertainty)
    - prior_covariance_scale (float, default = None): A-priori sigma_0^2     
    
    Return: 
    - res (nx1 np.ndarray): Vector of the weighted residuals
        
    '''    
        
    # Get parameters  
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    m = parvals['m']
    
    # Convert points to homogeneos coordinates and traspose np array to obtain a 4xn matrix
    x0 = convert_to_homogeneous(x0).T

    # Build 4x4 transformation matrix (T) in homogeneous coordinates
    T = np.identity(4)
    R = euler_matrix(rx, ry, rz)
    T[0:3, 0:3] = (m * np.identity(3)) @ R[:3,:3]
    T[0:3, 3:4] = np.array([tx, ty, tz]).reshape(3, 1) 
    
    # Apply transformation to x0 points
    x1_ = T @ x0
    x1_ = x1_[:3,:].T 

    # Compute residuals as differences between observed and estimated values, scaled by the a-priori observation uncertainties
    res = (x1 - x1_)
    
    # If weigthts are provided, scale residual
    if weights is not None:
        
        if weights.shape != res.shape:
            raise ValueError(
                f'Wrong dimensions of the weight matrix. It must be of size {res.shape}')
        
        res = res * weights
    
    return res.flatten()

In [19]:
# MAIN
pt_loc = read_data_to_df('../data/arsenale_loc.csv',
                         delimiter=';',
                         header=None,
                         col_names=['x','y','z'],
                         index_col=0,
                         )
pt_world = read_data_to_df('../data/arsenale_utm.csv',
                           delimiter=';',
                           header=None,
                           col_names=['x','y','z'],
                           index_col=0
                           )
pt_world
pt_loc

,x,y,z
100,961.938,965.543,100.249
200,918.310,978.849,100.335
300,896.649,1010.323,100.122
400,894.412,1080.206,99.969
500,974.617,1022.384,106.056
600,933.640,1028.106,106.989
700,995.258,1038.936,99.736
800,997.518,1006.376,99.977


In [22]:
from lib.transformations import affine_matrix_from_points
T = affine_matrix_from_points(pt_loc.to_numpy().T, pt_world.to_numpy().T,
                          shear=False, scale=True, usesvd=True)
t_ini = T[:3,:3:4].squeeze()
print(t_ini)
rot_ini = euler_from_matrix(T[:3,:3])
print(rot_ini)
m_ini = float(1.0)


[ 7.67983481e-01 -6.40087129e-01  4.17522578e-07]
(-4.4722575837373733e-07, -4.1762487502534233e-07, -0.6948158045093333)


In [23]:
# # Add camera centers
# cams_loc = pd.DataFrame(
#     {'x': [0., 254.0054], 'y': [0., -62.3993], 'z': [0.,  -5.0360]},
#     index=['p1', 'p2']
# )
# cams_world = pd.DataFrame(
#     {'x': [151.703, 312.930], 'y': [99.171, 300.536], 'z': [91.618, 135.159]},
#     index=['p1', 'p2']
# )
# pt_loc = pd.concat([pt_loc, cams_loc])
# pt_world = pd.concat([pt_world, cams_world])

# print(f'Points loc: \n {pt_loc}')
# print(f'Points world: \n {pt_world}')

# Initial values
# t_ini = np.array(
#     [552072, 4988733, 163],  # [1.46882746e+02, 8.74147624e+01, 9.04722323e+01]
#     dtype='float64'
# )
# rot_ini = np.array((0.01316323, 0, 0), 'float64')
# m_ini = float(1.0)

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot_ini[0], vary=True)
params.add('ry', value=rot_ini[1], vary=True)
params.add('rz', value=rot_ini[2], vary=True)
params.add('tx', value=t_ini[0], vary=True)
params.add('ty', value=t_ini[1], vary=True)
params.add('tz', value=t_ini[2], vary=True)
params.add('m',  value=m_ini, vary=True)

# Define Weights as the inverse of the a-priori standard deviation of each observation
# All the measurements are assumed as independent (Q diagonal)
# Weight matrix must have the same shape as the observation array X0
uncertainty = 2e-2 * np.ones(pt_loc.shape)  # Default assigned uncertainty[m]
uncertainty = pd.DataFrame(uncertainty)
uncertainty.columns = ['sx', 'sy', 'sz']
uncertainty.index = pt_loc.index
# uncertainty.loc['T2'] = [0.05, 0.05, 0.05]
# uncertainty.loc['T4'] = [0.05, 0.05, 0.05]
# uncertainty.loc['11'] = [0.2, 0.2, 0.2]
# uncertainty.loc['19'] = [0.2, 0.2, 0.2]
# uncertainty.loc['p1'] = [0.2, 0.2, 0.2]
# uncertainty.loc['p2'] = [0.2, 0.2, 0.2]
print(f'Prior uncertainties:\n{uncertainty}')

# A-priori Sigma_0²: scale of the covariance matrix
sigma0_2 = 1

# # Remove observations
# rows_to_drop = ['T2', 'T4', '11', '19']
# pt_loc = pt_loc.drop(labels = rows_to_drop)
# pt_world = pt_world.drop(labels = rows_to_drop)
# uncertainty = uncertainty.drop(labels=rows_to_drop)
# print(f'Points loc: \n {pt_loc}')

# Run Optimization!
weights = 1. / uncertainty.to_numpy()
minimizer = Minimizer(
    compute_residuals,
    params,
    fcn_args=(
        pt_loc.to_numpy(),
        pt_world.to_numpy(),
    ),
    fcn_kws={
        'weights': weights,
        'prior_covariance_scale': sigma0_2,
    },
    scale_covar=True,
)
result = minimizer.minimize(method='leastsq')
# fit_report(result)

# Print result
print_results(result, weights, sigma0_2)


Prior uncertainties:
       sx    sy    sz
100  0.02  0.02  0.02
200  0.02  0.02  0.02
300  0.02  0.02  0.02
400  0.02  0.02  0.02
500  0.02  0.02  0.02
600  0.02  0.02  0.02
700  0.02  0.02  0.02
800  0.02  0.02  0.02
-------------------------------
Optimization report
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 171
    # data points      = 24
    # variables        = 7
    chi-square         = 617024.490
    reduced chi-square = 36295.5582
    Akaike info crit   = 257.710644
    Bayesian info crit = 265.957021
[[Variables]]
    rx: -3.16409732 +/- 0.04140769 (1.31%) (init = -4.472258e-07)
    ry: -3.15477710 +/- 0.03541442 (1.12%) (init = -4.176249e-07)
    rz:  80.9864448 +/- 0.02642604 (0.03%) (init = -0.6948158)
    tx:  552072.511 +/- 36.6084217 (0.01%) (init = 0.7679835)
    ty:  4988733.30 +/- 36.6386907 (0.00%) (init = -0.6400871)
    tz:  163.506619 +/- 58.6975484 (35.90%) (init = 4.175226e-07)
    m:  -0.99389134 +/- 0.02622553 (2.64%) (init 